In [10]:
# librerias
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Parte 1: Preparación de Datos (15 puntos)

1.1 Cargue los datos en un DataFrame de Pandas

In [11]:
# cargar los datos a un dataframe
df = pd.read_csv('UK_foods.csv')
df

,Unnamed: 0,England,Wales,Scotland,N.Ireland
0,Cheese,105,103,103,66
1,Carcass_meat,245,227,242,267
2,Other_meat,685,803,750,586
3,Fish,147,160,122,93
4,Fats_and_oils,193,235,184,209
5,Sugars,156,175,147,139
6,Fresh_potatoes,720,874,566,1033
7,Fresh_Veg,253,265,171,143
8,Other_Veg,488,570,418,355
9,Processed_potatoes,198,203,220,187


1.2 Realice una exploración básica de los datos (estadísticas descriptivas).

In [13]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title='Food', explorative=True)
profile

ModuleNotFoundError: No module named 'pkg_resources'

In [ ]:
from sklearn.preprocessing import MinMaxScaler
foods = df['Food']
data = df.drop('Food', axis=1)

scaler = MinMaxScaler()
data = scaler.fit_transform(data)
data = pandas.DataFrame(data, columns=df.columns[1:])
data.head()